In [1]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import imutils
import cv2

Using TensorFlow backend.


In [2]:
CASCADE = "haarcascade_frontalface_default.xml"
MODEL = "output/lenet.hdf5"

In [3]:
detector = cv2.CascadeClassifier(CASCADE)
model = load_model(MODEL)

In [11]:
# IF a video path was not supplied grab the reference to the webcam
#if not args.get("video", False):
camera = cv2.VideoCapture(0)
#else:
#    camera = cv2.VideoCapture("VIDEO")

In [12]:
# Keep looping
while True:
    # Grab the current frame
    (grabbed, frame) = camera.read()
    
    # If we are viewing a video and we did not grab a frame, then we
    # have reached the end of the video
    if not grabbed:
        break
        
    # REsize the frame, conver it to grayscale and ten clone the original frame
    # so we can draw on it later in the program
    frame = imutils.resize(frame, width=300)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frameClone = frame.copy()
    
    # Detect faces in the input frame, then clone the frame so that we can draw on it
    rects = detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE)
    
    # Loop over the face bounding boxes
    for(fX, fY, fW, fH) in rects:
        # Extract the ROI of the face from the grayscale image
        # resize it to a fixed 28x28 pixels and then prepare the ROI
        # for classification via the CNN
        roi = gray[fY:fY+fH, fX:fX + fW]
        roi = cv2.resize(roi, (28, 28))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        
        # Determine the probabilities of both smiling and not smiling 
        # then set the label accordingly
        (notSmiling, smiling) = model.predict(roi)[0]
        label = "Smiling" if smiling > notSmiling else "Not Smiling"
        
        # Display the labe and bounding box rectangle on the output frame
        cv2.putText(frameClone, label, (fX, fY - 10), 
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        cv2.rectangle(frameClone, (fX, fY), (fX+fW, fY+fH), (0, 0, 255), 2)
        
    # Show our detected faces along with smiling/not smiling labels
    cv2.imshow("Face", frameClone)
    
    # If the q key is pressed stop the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
camera.release()
cv2.destroyAllWindows()
    

In [13]:
camera.release()























